In [1]:
from __future__ import print_function
import os
import sys

In [2]:
# Someday this has to go in a config or kernel

spark_home = '/opt/spark/latest'

os.environ['SPARK_HOME'] = spark_home
sys.path.insert(0, spark_home + "/python")
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.1-src.zip'))

from pyspark.context import SparkContext
from pyspark.context import SparkConf
from pyspark.sql import SQLContext



https://www.elastic.co/guide/en/elasticsearch/hadoop/current/configuration.html

It looks like we need to help out the mapper, es.field.read.as.array.include. How and where to set this?

https://spark.apache.org/docs/1.5.0/api/python/_modules/pyspark/conf.html
http://stackoverflow.com/questions/32362783/how-to-change-sparkcontext-properties-in-interactive-pyspark-session

Looks like making a conf object before creating the context. Can we really not change this at runtime?
"
once a SparkConf object is passed to Spark, it is cloned and can no longer be modified by the user. Spark does not support modifying the configu$
"


In [3]:
# The conf is imutable so the config must be created first and then passed in. This 
# has to be re-run whenever the config is changed.

conf = SparkConf()
conf.set("driver-class-path",
        "/home/mcollins/elasticsearch-hadoop-2.4.0/dist/elasticsearch-spark_2.11-2.4.0.jar")
conf.set("es.nodes", 
         "c18node14.acis.ufl.edu,c18node2.acis.ufl.edu,c18node6.acis.ufl.edu,c18node10.acis.ufl.edu,c18node12.acis.ufl.edu")
conf.set("es.read.field.as.array.include",
        "indexData")

# No combination of the options "es.read.field.as.array.include", "ES_READ_FIELD_AS_ARRAY_INCLUDE", 
# "gbif:vernacularname" and "indexData.gbif:vernacularname" seems to make a difference in the 
# schema that is read, it is always struct (actually I was using an older es.field.read... [note reversed words] 
# flag which was changed last year.)
#
# Now using "indexData.gbif:vernacularname.dcterms:source" and "es.read.field.as.array.include" yeilds an 
# exception about not being able to read things as arrays so that is the right syntax at least


# This doesn't work, stop and start the kernel instead.
try:
    sc
except:
    pass
else:
    sc.stop()
    sc = None
    sqlContext = None
    
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

About the ES connector: version 2.1 of the elasticsearch-hadoop connector (not 2.1 of spark!) has native support for Spark now, released around June 2016. https://www.elastic.co/guide/en/elasticsearch/hadoop/current/spark.html

Video (fake registration works, real content @ about 9:00) https://www.elastic.co/webinars/adding-spark-and-security-to-elasticsearch-for-hadoop/?baymax=rtp&elektra=docs&iesrc=ctr

In [4]:
es_df = es_df = sqlContext.read.format("org.elasticsearch.spark.sql").load("idigbio-2.10.2/records")
es_df.printSchema()

Py4JJavaError: An error occurred while calling o29.load.
: org.elasticsearch.hadoop.EsHadoopIllegalArgumentException: Invalid parameter [indexData.gbif:vernacularname] specified in setting [indexData.gbif:vernacularname]
	at org.elasticsearch.hadoop.util.SettingsUtils.getFieldArrayFilterInclude(SettingsUtils.java:201)
	at org.elasticsearch.spark.sql.SchemaUtils$.convertToStruct(SchemaUtils.scala:106)
	at org.elasticsearch.spark.sql.SchemaUtils$.discoverMapping(SchemaUtils.scala:70)
	at org.elasticsearch.spark.sql.ElasticsearchRelation.lazySchema$lzycompute(DefaultSource.scala:112)
	at org.elasticsearch.spark.sql.ElasticsearchRelation.lazySchema(DefaultSource.scala:112)
	at org.elasticsearch.spark.sql.ElasticsearchRelation$$anonfun$schema$1.apply(DefaultSource.scala:116)
	at org.elasticsearch.spark.sql.ElasticsearchRelation$$anonfun$schema$1.apply(DefaultSource.scala:116)
	at scala.Option.getOrElse(Option.scala:121)
	at org.elasticsearch.spark.sql.ElasticsearchRelation.schema(DefaultSou

The zip file here has all the jars in it: https://www.elastic.co/downloads/hadoop

And example code:

https://spark.apache.org/docs/latest/programming-guide.html#external-datasets

Full docs, only Scala and Java samples except the last lines which are Python:

https://www.elastic.co/guide/en/elasticsearch/hadoop/current/spark.html


ls /opt/spark/latest/jars/ela*
/opt/spark/latest/jars/elasticsearch-hadoop-2.4.0.jar

https://db-blog.web.cern.ch/blog/prasanth-kothuri/2016-05-integrating-hadoop-and-elasticsearch-%E2%80%93-part-2-%E2%80%93-writing-and-querying


```
>>> es_df = sqlContext.read.format("org.elasticsearch.spark.sql").load("idigbio-2.10.1/records")
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "/opt/spark/latest/python/pyspark/sql/readwriter.py", line 147, in load
    return self._df(self._jreader.load(path))
  File "/opt/spark/latest/python/lib/py4j-0.10.1-src.zip/py4j/java_gateway.py", line 933, in __call__
  File "/opt/spark/latest/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/opt/spark/latest/python/lib/py4j-0.10.1-src.zip/py4j/protocol.py", line 312, in get_return_value
py4j.protocol.Py4JJavaError: An error occurred while calling o31.load.
: org.elasticsearch.hadoop.EsHadoopIllegalArgumentException: Unknown GeoShape [null]
        at org.elasticsearch.hadoop.serialization.dto.mapping.MappingUtils.doParseGeoShapeInfo(MappingUtils.java:254)
        at org.elasticsearch.hadoop.serialization.dto.mapping.MappingUtils.parseGeoInfo(MappingUtils.java:219)
        at org.elasticsearch.hadoop.rest.RestRepository.sampleGeoFields(RestRepository.java:447)
        at org.elasticsearch.spark.sql.SchemaUtils$.discoverMappingAsField(SchemaUtils.scala:86)
        at org.elasticsearch.spark.sql.SchemaUtils$.discoverMapping(SchemaUtils.scala:69)
        at org.elasticsearch.spark.sql.ElasticsearchRelation.lazySchema$lzycompute(DefaultSource.scala:112)
        at org.elasticsearch.spark.sql.ElasticsearchRelation.lazySchema(DefaultSource.scala:112)
        at org.elasticsearch.spark.sql.ElasticsearchRelation$$anonfun$schema$1.apply(DefaultSource.scala:116)
        at org.elasticsearch.spark.sql.ElasticsearchRelation$$anonfun$schema$1.apply(DefaultSource.scala:116)
        at scala.Option.getOrElse(Option.scala:121)
        at org.elasticsearch.spark.sql.ElasticsearchRelation.schema(DefaultSource.scala:116)
        at org.apache.spark.sql.execution.datasources.LogicalRelation.<init>(LogicalRelation.scala:40)
        at org.apache.spark.sql.SparkSession.baseRelationToDataFrame(SparkSession.scala:382)
        at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:143)
        at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:132)
        at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
        at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
        at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
        at java.lang.reflect.Method.invoke(Method.java:498)
        at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:237)
        at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
        at py4j.Gateway.invoke(Gateway.java:280)
        at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:128)
        at py4j.commands.CallCommand.execute(CallCommand.java:79)
        at py4j.GatewayConnection.run(GatewayConnection.java:211)
        at java.lang.Thread.run(Thread.java:745)
```

https://github.com/elastic/elasticsearch-hadoop/issues/607

Alex took out the GeoShape field in the 2.10.2 index because it was all nulls and unused. The documented behavior when converting this field type from ES to a DF is to sample one row and use that to determine the data strucuture. It's not happy when all rows are null apparently.

https://www.elastic.co/guide/en/elasticsearch/hadoop/current/mapping.html


In [5]:
es_df.head(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.sql.execution.python.EvaluatePython.takeAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, localhost): org.elasticsearch.hadoop.EsHadoopIllegalStateException: Field 'indexData.gbif:vernacularname.dcterms:source' not found; typically this occurs with arrays which are not mapped as single value
	at org.elasticsearch.spark.sql.RowValueReader$class.rowColumns(RowValueReader.scala:34)
	at org.elasticsearch.spark.sql.ScalaRowValueReader.rowColumns(ScalaEsRowValueReader.scala:14)
	at org.elasticsearch.spark.sql.ScalaRowValueReader.createMap(ScalaEsRowValueReader.scala:51)
	at org.elasticsearch.hadoop.serialization.ScrollReader.map(ScrollReader.java:778)
	at org.elasticsearch.hadoop.serialization.ScrollReader.read(ScrollReader.java:704)
	at org.elasticsearch.hadoop.serialization.ScrollReader.list(ScrollReader.j